In [1]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
# import squidpy as sq

from scipy.spatial import cKDTree
import mudata as mu

import spatialdata as sd
import anndata as an

In [2]:
## load slide
slide = "B1"
exp = "V11L12-109" # V11T17-102 V11L12-109 V11T16-085
key = f"{exp}_{slide}"
slide_path = f"data/VicariEtAl/sma/{exp}/{key}/output_data"

In [3]:
mdata = mu.read_h5mu(os.path.join(slide_path, "sma.h5mu"))
rna = mdata.mod["rna"]
msi = mdata.mod["msi"]

/home/dbdimitrov/anaconda3/envs/spatialdata/lib/python3.10/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/dbdimitrov/anaconda3/envs/spatialdata/lib/python3.10/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/dbdimitrov/anaconda3/envs/spatialdata/lib/python3.10/site-packages/anndata/_core/anndata.py:1900: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/dbdimitrov/anaconda3/envs/spatialdata/lib/python3.10/site-packages/mudata/_core/mudata.py:479: UserWarning: var_names are not unique. To make them unique, call `.var_names_make_unique`.
  warnings.warn(


In [4]:
rna_pos = rna.obsm['spatial'].copy()
msi_pos = msi.obsm['spatial'].copy()

In [5]:
from spatialdata.models import TableModel, ShapesModel
from anndata import AnnData
import numpy as np
from spatialdata import SpatialData

In [6]:
# shapes 
rna_locations = ShapesModel.parse(rna_pos, geometry=0, radius=1)
msi_locations = ShapesModel.parse(msi_pos, geometry=0, radius=1)

try spatialdata with multiple anndatas

In [7]:
del msi.varm['PCs']

In [8]:
adata = an.concat([rna, msi], join='outer', axis=1, label='modality')
adata.var_names_make_unique()

/home/dbdimitrov/anaconda3/envs/spatialdata/lib/python3.10/site-packages/anndata/_core/anndata.py:1900: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [9]:
adata.obs['region'] = (['msi'] * msi_pos.shape[0] + ['rna'] * rna_pos.shape[0]).copy()
adata.obs['instance_id'] = adata.obs.index

In [10]:
adata.obs

,region,instance_id
1000,msi,1000
1001,msi,1001
1002,msi,1002
1003,msi,1003
1004,msi,1004
...,...,...
TTGTTCAGTGTGCTAC-1,rna,TTGTTCAGTGTGCTAC-1
TTGTTCTAGATACGCT-1,rna,TTGTTCTAGATACGCT-1
TTGTTGTGTGTCAAGA-1,rna,TTGTTGTGTGTCAAGA-1
TTGTTTCCATACAACT-1,rna,TTGTTTCCATACAACT-1


In [11]:
adata = TableModel.parse(adata, region=['msi', 'rna'], region_key='region', instance_key='instance_id')
sdata = SpatialData(shapes={'msi':msi_locations, 'rna':rna_locations},
                    table=adata,
                    # TODO: add images (need to read visium as sdata - would be easiest)
                    #                 images={
                    #     "CytAssist_FFPE_Human_Breast_Cancer_full_image": rna.uns['spatial'][key]['images']['hires']
                    # # },
                    )

/home/dbdimitrov/anaconda3/envs/spatialdata/lib/python3.10/site-packages/spatialdata/models/models.py:725: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(adata.obs[region_key]):
/tmp/ipykernel_17068/1232050206.py:1: UserWarning: Converting `region_key: region` to categorical dtype.
  adata = TableModel.parse(adata, region=['msi', 'rna'], region_key='region', instance_key='instance_id')


In [12]:
sdata

SpatialData object with:
├── Shapes
│     ├── 'msi': GeoDataFrame shape: (3005, 2) (2D shapes)
│     └── 'rna': GeoDataFrame shape: (3120, 2) (2D shapes)
└── Table
      └── AnnData object with n_obs × n_vars = 6125 × 33823
    obs: 'region', 'instance_id'
    var: 'gene_ids', 'feature_types', 'genome', 'mean', 'std', 'modality'
    uns: 'spatialdata_attrs': AnnData (6125, 33823)
with coordinate systems:
▸ 'global', with elements:
        msi (Shapes), rna (Shapes)

In [15]:
sdata.shapes['rna']

,geometry,radius
0,POINT (34740.000 30651.000),1
1,POINT (7784.000 27046.000),1
2,POINT (33275.000 36197.000),1
3,POINT (38516.000 28139.000),1
4,POINT (41113.000 32686.000),1
...,...,...
3115,POINT (23757.000 17504.000),1
3116,POINT (6074.000 15942.000),1
3117,POINT (27517.000 21045.000),1
3118,POINT (13000.000 28069.000),1


In [18]:
sdata.table.obs

,region,instance_id
1000,msi,1000
1001,msi,1001
1002,msi,1002
1003,msi,1003
1004,msi,1004
...,...,...
TTGTTCAGTGTGCTAC-1,rna,TTGTTCAGTGTGCTAC-1
TTGTTCTAGATACGCT-1,rna,TTGTTCTAGATACGCT-1
TTGTTGTGTGTCAAGA-1,rna,TTGTTGTGTGTCAAGA-1
TTGTTTCCATACAACT-1,rna,TTGTTTCCATACAACT-1


Try as MuData

In [19]:
sdata

SpatialData object with:
├── Shapes
│     ├── 'msi': GeoDataFrame shape: (3005, 2) (2D shapes)
│     └── 'rna': GeoDataFrame shape: (3120, 2) (2D shapes)
└── Table
      └── AnnData object with n_obs × n_vars = 6125 × 33823
    obs: 'region', 'instance_id'
    var: 'gene_ids', 'feature_types', 'genome', 'mean', 'std', 'modality'
    uns: 'spatialdata_attrs': AnnData (6125, 33823)
with coordinate systems:
▸ 'global', with elements:
        msi (Shapes), rna (Shapes)

In [ ]:
sdata = SpatialData(shapes={'msi':msi_locations, 'rna':rna_locations},
                    table=adata,
                    # TODO: add images (need to read visium as sdata - would be easiest)
                    #                 images={
                    #     "CytAssist_FFPE_Human_Breast_Cancer_full_image": rna.uns['spatial'][key]['images']['hires']
                    # # },
                    )